# 

## Install Python libraries
* To temporarily install Python libraries for server use, use the command: `%pip`.

In [12]:
%pip install simplekml
%pip install geopy
%pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Python libraries

In [13]:
from sliderule import sliderule, icesat2, earthdata
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from shapely.geometry import Polygon, Point, mapping
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.linear_model import LinearRegression
import geopy
import simplekml
from geopy.distance import geodesic
from matplotlib.backends.backend_pdf import PdfPages
import plotly.express as px
from tqdm import tqdm
import warnings

# Define the Region:
define the region using the toregion function, which converts the GeoJSON file into a format recognized by SlideRule.

In [14]:
# Use sliderule toregion function to define the region
region = sliderule.toregion('geojson_files/BONA_buffer_8km.geojson')

# Set Parameters:

In [15]:
site_name= "BONA"
segment_meter = "100"
cnt = "10"
ats = "50"

## Time Parameter

In [16]:
%%time
# Configuration settings
year = 2021
month = 10
day = 19

# Generate start_time and end_time dynamically
def generate_time_range(year, month, day):
    """
    Generate start and end time for a given year, month, and day.

    Args:
        year (int): The year (e.g., 2023).
        month (int): The month (1-12).
        day (int): The day (default: 1).

    Returns:
        tuple: Start time and end time strings in ISO 8601 format.
    """
    start_time = datetime(year, month, day).strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time = datetime(year, month, day).replace(hour=23, minute=59, second=59).strftime("%Y-%m-%dT%H:%M:%SZ")
    return start_time, end_time

# Use the configuration settings to generate start_time and end_time
start_time, end_time = generate_time_range(year, month, day)

# Print results for verification
print(f"Start Time: {start_time}")
print(f"End Time: {end_time}")


Start Time: 2021-10-19T00:00:00Z
End Time: 2021-10-19T23:59:59Z
CPU times: user 295 µs, sys: 0 ns, total: 295 µs
Wall time: 265 µs


# Build ATL03 Request Parameters:

Construct a dictionary of parameters to specify how the ATL03 data should be processed.

In [17]:
%%time
parms = {
    "poly": region['poly'],# Polygon defining the spatial region of interest
    "t0": start_time, # Start time of the data collection period (ISO 8601 format)
    "t1": end_time, # End time of the data collection period (ISO 8601 format)
    "srt": icesat2.SRT_LAND, # Surface return type: specifies the data type (e.g., LAND, ICE, etc.)
    "cnf": 0, # Confidence level for photon classification (e.g., 0: all photons)
    "cnt": cnt, # Number of photons required for the analysis (optional)
    "ats": ats, # Along-track spacing for the data (optional, controls density of data)
    "len": segment_meter,# Segment length in meters, used for segmentation 
    "res": segment_meter, # Resolution of the output, in meters
    # List of ATL08 photon classifications to include in the data
    "atl08_class": [
        "atl08_noise",        # Photons classified as noise
        "atl08_ground",       # Ground-level photons
        "atl08_canopy",       # Photons classified as canopy
        "atl08_top_of_canopy",# Photons classified as the top of the canopy
        "atl08_unclassified"  # Unclassified photons
    ],
    # Phoreal settings for processing photon data
    "phoreal": {
        "binsize": 1.0,          # Bin size for photon aggregation (e.g., 1 meter)
        "geoloc": "mean",        # Method for geolocation aggregation (e.g., mean)
        "use_abs_h": True,       # Whether to use absolute heights in calculations
        "send_waveform": True    # Whether to include waveform data in the output
    }
}


CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 18.1 µs


# Retrieve ATL03 Data:
Use the atl03sp function to retrieve the ATL03 data based on the specified parameters.

In [18]:
%%time
# Retrieve ATL03 data, retaining the extent ID
atl03_data = icesat2.atl03sp(parms, keep_id=True)

CPU times: user 2.62 s, sys: 75.8 ms, total: 2.7 s
Wall time: 4.13 s


# Data processing on ATL03 photon data
* extracting coordinates, separating photon classes, and counting canopy and terrain photons by segment.

In [19]:
%%time
# Ensure 'time' is the index and in datetime format
atl03_data.index = pd.to_datetime(atl03_data.index)

# Extract latitude and longitude from the 'geometry' column
atl03_data['latitude'] = atl03_data['geometry'].apply(lambda x: x.y if x else None)
atl03_data['longitude'] = atl03_data['geometry'].apply(lambda x: x.x if x else None)

# Separate photons into canopy and terrain classes
canopy_photons = atl03_data[atl03_data['atl08_class'].isin([2, 3])]  # Classes 2 and 3 for canopy
terrain_photons = atl03_data[atl03_data['atl08_class'] == 1]         # Class 1 for ground

# Count photons for each segment
canopy_counts = canopy_photons.groupby('extent_id').size().reset_index(name='canopy_photon_count')
terrain_counts = terrain_photons.groupby('extent_id').size().reset_index(name='terrain_photon_count')

# Calculate unique time values for each segment with include_groups=False to avoid the warning
unique_time_count = (
    atl03_data.groupby('extent_id', group_keys=False)
    .apply(lambda x: x.index.nunique())
    .reset_index(name='unique_shots')
)
# Step 1: Merge canopy and terrain counts into one DataFrame
segment_counts = (
    canopy_counts
    .merge(terrain_counts, on='extent_id', how='outer')
    .merge(unique_time_count, on='extent_id', how='outer')
)

# Step 2: Replace any zero or NaN values in 'unique_shots' with NaN to avoid division by zero errors
segment_counts['unique_shots'].replace(0, np.nan, inplace=True)

# Step 3: Calculate photon rates by normalizing with the number of unique shots
segment_counts['canopy_photon_rate'] = segment_counts['canopy_photon_count'] / segment_counts['unique_shots']
segment_counts['terrain_photon_rate'] = segment_counts['terrain_photon_count'] / segment_counts['unique_shots']

# Step 5: Reset index to keep 'time' as a column for merging
atl03_data = atl03_data.reset_index()
# Merge photon rates into the main DataFrame
atl03_data = atl03_data.merge(segment_counts[['extent_id', 'canopy_photon_rate', 'terrain_photon_rate']], 
                              on='extent_id', how='left')

CPU times: user 3.29 s, sys: 13.3 ms, total: 3.31 s
Wall time: 3.33 s


<timed exec>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
<timed exec>:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




### Landcover Assignment
Calculate the mode (most frequent value) of landcover per `extent_id`, while handling missing or invalid values.

In [21]:
%%time
# Step 1: Ensure there are no invalid landcover values before grouping
valid_landcover_codes = {111, 113, 112, 114, 115, 116, 121, 123, 122, 124, 125, 126, 20}

# Filter out rows with invalid or missing landcover values
valid_landcover_data = atl03_data[atl03_data["landcover"].isin(valid_landcover_codes)].copy()

# Step 2: Calculate the landcover mode for each extent_id
landcover_modes = (
    valid_landcover_data.groupby("extent_id")["landcover"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
)

# Step 3: Assign the calculated mode back to the original DataFrame
atl03_data["landcover_mode"] = atl03_data["extent_id"].map(landcover_modes)


# Step 4: Set 'time' back as the index
atl03_data.set_index("time", inplace=True)

# Ensure 'time' is in datetime format
atl03_data.index = pd.to_datetime(atl03_data.index)

# Step 5: Extract month, year, and day for aggregation
atl03_data["month"] = atl03_data.index.month
atl03_data["year"] = atl03_data.index.year
atl03_data["day"] = atl03_data.index.day

# Display the updated DataFrame
atl03_data.head(50)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/IPython/core/magics/execution.py", line 1340, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 17, in <module>
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/pandas/core/frame.py", line 6122, in set_index
    raise KeyError(f"None of {missing} are in the columns")
KeyError: "None of ['time'] are in the columns"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2142, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/srv

In [28]:
# Define the range
# start_time = pd.Timestamp("2021-10-19 10:06:00.000000000")
# end_time = pd.Timestamp("2021-10-19 11:07:00.000000000")

# filtered_df = atl03_data[(atl03_data.index >= start_time) & (atl03_data.index <= end_time)]

atl03_data.head(50)

,region,track,solar_elevation,segment_dist,segment_id,sc_orient,pair,cycle,background_rate,extent_id,...,geometry,spot,latitude,longitude,canopy_photon_rate,terrain_photon_rate,landcover_mode,month,year,day
time,,,,,,,,,,,,,,,,,,,,,
2021-10-19 09:06:46.157715200,5,3,-34.703072,1.280433e+07,639265,0,0,13,30638.645554,1850980364898533376,...,POINT (-147.44191 65.18992),5,65.189916,-147.441910,1.741007,2.086331,116,10,2021,19
2021-10-19 09:06:46.157715200,5,3,-34.703072,1.280433e+07,639265,0,0,13,30638.645554,1850980364898533376,...,POINT (-147.44191 65.18992),5,65.189916,-147.441910,1.741007,2.086331,116,10,2021,19
2021-10-19 09:06:46.157715200,5,3,-34.703072,1.280433e+07,639265,0,0,13,30638.645554,1850980364898533376,...,POINT (-147.44191 65.18992),5,65.189916,-147.441910,1.741007,2.086331,116,10,2021,19
2021-10-19 09:06:46.157715200,5,3,-34.703072,1.280433e+07,639265,0,0,13,30638.645554,1850980364898533376,...,POINT (-147.4419 65.18992),5,65.189916,-147.441904,1.741007,2.086331,116,10,2021,19
2021-10-19 09:06:46.157715200,5,3,-34.703072,1.280433e+07,639265,0,0,13,30638.645554,1850980364898533376,...,POINT (-147.44191 65.18992),5,65.189916,-147.441910,1.741007,2.086331,116,10,2021,19
2021-10-19 09:06:46.157715200,5,3,-34.703072,1.280433e+07,639265,0,0,13,30638.645554,1850980364898533376,...,POINT (-147.44191 65.18992),5,65.189916,-147.441911,1.741007,2.086331,116,10,2021,19
2021-10-19 09:06:46.157715200,5,3,-34.703072,1.280433e+07,639265,0,0,13,30638.645554,1850980364898533376,...,POINT (-147.44191 65.18992),5,65.189916,-147.441911,1.741007,2.086331,116,10,2021,19
2021-10-19 09:06:46.157715200,5,3,-34.703072,1.280433e+07,639265,0,0,13,30638.645554,1850980364898533376,...,POINT (-147.44191 65.18992),5,65.189916,-147.441910,1.741007,2.086331,116,10,2021,19
2021-10-19 09:06:46.157715200,5,3,-34.703072,1.280433e+07,639265,0,0,13,30638.645554,1850980364898533376,...,POINT (-147.44191 65.18992),5,65.189916,-147.441910,1.741007,2.086331,116,10,2021,19


In [22]:
atl03_data.to_csv("atl03_data.csv")

In [ ]:
# Ensure 'time' is in datetime format and extract the date part
atl03_data['date'] = atl03_data.index.date

# Group by date to count ground photons (atl08_class == 1)
ground_photon_counts = atl03_data[atl03_data['atl08_class'] == 1].groupby('date').size().reset_index(name='ground_photon_count')

# Group by date to count canopy photons (atl08_class in [2, 3])
canopy_photon_counts = atl03_data[atl03_data['atl08_class'].isin([2, 3])].groupby('date').size().reset_index(name='canopy_photon_count')

# Group by date and count unique segments (extent_id) for each date
segment_counts_by_date = atl03_data.groupby('date')['extent_id'].nunique().reset_index(name='segment_count')

# Merge ground, canopy, and segment counts into a single DataFrame
counts_by_date = ground_photon_counts.merge(canopy_photon_counts, on='date', how='outer')
counts_by_date = counts_by_date.merge(segment_counts_by_date, on='date', how='outer')

# Fill NaN values with 0 for days where no ground or canopy photons are present
counts_by_date.fillna(0, inplace=True)

# Display the unique dates, ground photon counts, canopy photon counts, and segment counts
print(counts_by_date)

# DATA Visualization

In [ ]:
# Define the landcover color map and descriptions
landcover_colors = {
    111: '#67000d',  # Closed forest, evergreen needle leaf
    113: '#a50f15',  # Closed forest, deciduous needle leaf
    112: '#cb181d',  # Closed forest, evergreen broad leaf
    114: '#ef3b2c',  # Closed forest, deciduous broad leaf
    115: '#fb6a4a',  # Closed forest, mixed
    116: '#fc9272',  # Closed forest, unknown
    121: '#00441b',  # Open forest, evergreen needle leaf
    123: '#006d2c',  # Open forest, deciduous needle leaf
    122: '#238b45',  # Open forest, evergreen broad leaf
    124: '#41ab5d',  # Open forest, deciduous broad leaf
    125: '#74c476',  # Open forest, mixed
    126: '#a1d99b',  # Open forest, unknown
    20: '#6baed6'    # Shrubs
}

landcover_descriptions = {
    111: 'Closed forest, evergreen needle leaf',
    113: 'Closed forest, deciduous needle leaf',
    112: 'Closed forest, evergreen broad leaf',
    114: 'Closed forest, deciduous broad leaf',
    115: 'Closed forest, mixed',
    116: 'Closed forest, unknown',
    121: 'Open forest, evergreen needle leaf',
    123: 'Open forest, deciduous needle leaf',
    122: 'Open forest, evergreen broad leaf',
    124: 'Open forest, deciduous broad leaf',
    125: 'Open forest, mixed',
    126: 'Open forest, unknown',
    20: 'Shrubs'
}

In [ ]:
import matplotlib.pyplot as plt

def plot_photon_rates_for_month(atl03_data, month, year, x_min, x_max, y_min, y_max, landcover_colors):
    """
    Plots terrain_photon_rate vs. canopy_photon_rate for a specific month, with points colored by landcover.

    Parameters:
    - atl03_data: DataFrame containing photon data with a 'month' column.
    - month: int, the month to plot.
    - year: int, the year for the title.
    - x_min, x_max: float, x-axis range for terrain photon rate.
    - y_min, y_max: float, y-axis range for canopy photon rate.
    - landcover_colors: dict, mapping of landcover codes to colors.
    """
    # Filter data for the specified month
    monthly_data = atl03_data[atl03_data['month'] == month]

    # Set a default color for undefined landcover values
    default_color = '#d9d9d9'  # Light gray for undefined landcover

    # Map landcover to colors, using the default color for NaN values
    monthly_data['color'] = monthly_data['landcover_mode'].map(landcover_colors).fillna(default_color)

    # Create the scatter plot
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(
        monthly_data['terrain_photon_rate'],
        monthly_data['canopy_photon_rate'],
        c=monthly_data['color'],  # Use the mapped colors for landcover
        alpha=0.8,
        edgecolor='w',
        s=20  # Point size
    )
    
    # Set plot title and labels
    plt.title(f'Terrain Photon Rate vs. Canopy Photon Rate for Month {month}, {year}')
    plt.xlabel('Terrain Photon Rate')
    plt.ylabel('Canopy Photon Rate')

    # Set axis limits
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    # Add a custom legend for landcover types
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', label=desc, markerfacecolor=color, markersize=8)
        for lc, color in landcover_colors.items()
        for key, desc in landcover_descriptions.items() if lc == key
    ]
    legend_elements.append(
        plt.Line2D([0], [0], marker='o', color='w', label='Undefined Landcover', markerfacecolor=default_color, markersize=8)
    )
    
    # Add the legend outside the plot
    plt.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5), title='Landcover Types', fontsize='small')

    # Adjust layout to make space for the legend
    plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust right space to make room for legend

    # Show the plot
    plt.show()

# Example usage: plot photon rates for April with landcover coloring
plot_photon_rates_for_month(atl03_data, month, year, x_min=0, x_max=8, y_min=0, y_max=12, landcover_colors=landcover_colors)


# By Segment Data Preparation

## Unique Segment Extraction:

In [ ]:
# Filter for ground and canopy photons (atl08_class 1, 2, and 3)
atl03_data = atl03_data[atl03_data['atl08_class'].isin([1, 2, 3])]

# Extent Identification
# Extract start and end coordinates for each unique extent_id
atl03_data['lat_start'] = atl03_data.groupby('extent_id')['latitude'].transform('first')
atl03_data['lon_start'] = atl03_data.groupby('extent_id')['longitude'].transform('first')
atl03_data['lat_end'] = atl03_data.groupby('extent_id')['latitude'].transform('last')
atl03_data['lon_end'] = atl03_data.groupby('extent_id')['longitude'].transform('last')

# Get unique extents with relevant information
unique_extents = atl03_data[['track', 'rgt', 'extent_id', 'lat_start', 'lon_start', 
                             'lat_end', 'lon_end', 'month']].drop_duplicates()


### Creating a Folium Map of Segment Start and End Points, htlm, csv and kml file

In [ ]:
import folium
from folium.plugins import FeatureGroupSubGroup
from tqdm import tqdm
import pandas as pd
import simplekml


def create_interactive_map_with_kml_and_csv(
    atl03_data, site_name, year, landcover_colors, output_file, csv_file, kml_file, sample_frac=0.1
):
    """
    Create an interactive Folium map, export a CSV summary, and generate a KML file with extent polygons.

    Args:
        atl03_data (DataFrame): The ATL03 data containing extents and their landcover modes.
        site_name (str): The name of the site for saving the output file.
        year (int): The year for saving the output file.
        landcover_colors (dict): A dictionary mapping landcover codes to colors.
        output_file (str): The name of the output HTML file for the map.
        csv_file (str): The name of the output CSV file summarizing extent information.
        kml_file (str): The name of the output KML file summarizing extent polygons.
        sample_frac (float): Fraction of the data to sample (default: 0.1, i.e., 10%).
    """
    # Sample the dataset to reduce the number of rows (optional)
    atl03_data = atl03_data.sample(frac=sample_frac, random_state=42)
    print(f"Processing {len(atl03_data)} extents...")

    # Calculate the center of the map
    map_center = [atl03_data['latitude'].mean(), atl03_data['longitude'].mean()]
    folium_map = folium.Map(location=map_center, zoom_start=12)

    # Create a FeatureGroup for each unique landcover type
    base_group = folium.FeatureGroup(name="All Extents", show=True).add_to(folium_map)

    landcover_groups = {}
    for landcover, color in landcover_colors.items():
        landcover_name = f"Landcover: {landcover}"
        group = FeatureGroupSubGroup(base_group, name=landcover_name)
        landcover_groups[landcover] = group
        folium_map.add_child(group)

    # Add a default group for 'Unknown' landcover
    if 'Unknown' not in landcover_colors:
        landcover_groups['Unknown'] = FeatureGroupSubGroup(base_group, name="Landcover: Unknown")
        folium_map.add_child(landcover_groups['Unknown'])

    # Prepare lists for CSV and KML export
    csv_rows = []
    kml = simplekml.Kml()

    # Add extents to the map with landcover color
    for _, row in tqdm(atl03_data.iterrows(), total=len(atl03_data), desc="Processing extents"):
        extent_coords = [(row['lat_start'], row['lon_start']), (row['lat_end'], row['lon_end'])]
        landcover = row['landcover_mode']
        landcover_color = landcover_colors.get(landcover, "#000000")  # Default to black if undefined

        # Add extent polygon to Folium map
        folium.Polygon(
            locations=extent_coords,
            color=landcover_color,
            weight=2,
            fill=True,
            fill_opacity=0.5,
            tooltip=f"Extent ID: {row['extent_id']}<br>Landcover: {landcover}",
        ).add_to(landcover_groups.get(landcover, landcover_groups['Unknown']))

        # Add extent polygon to KML file
        polygon = kml.newpolygon(
            name=f"Extent {row['extent_id']}",
            outerboundaryis=[(lon, lat) for lat, lon in extent_coords] + [(row['lon_start'], row['lat_start'])],
        )
        polygon.style.polystyle.color = simplekml.Color.changealphaint(100, landcover_color.lstrip("#"))
        polygon.style.linestyle.color = simplekml.Color.changealphaint(100, landcover_color.lstrip("#"))
        polygon.style.linestyle.width = 2
        polygon.description = f"Extent ID: {row['extent_id']}\nLandcover: {landcover}"

        # Add data for CSV export
        csv_rows.append({
            "extent_id": row['extent_id'],
            "landcover_mode": landcover,
            "lat_start": row['lat_start'],
            "lon_start": row['lon_start'],
            "lat_end": row['lat_end'],
            "lon_end": row['lon_end'],
        })

    # Export the CSV file
    csv_df = pd.DataFrame(csv_rows)
    csv_df.to_csv(csv_file, index=False)
    print(f"CSV file saved as {csv_file}")

    # Save the KML file
    kml.save(kml_file)
    print(f"KML file saved as {kml_file}")

    # Add a LayerControl for toggling visibility
    folium.LayerControl(collapsed=False).add_to(folium_map)

    # Save the map to an HTML file
    folium_map.save(output_file)
    print(f"Interactive map saved as {output_file}")


# Example usage
create_interactive_map_with_kml_and_csv(
    atl03_data,
    site_name,
    year,
    landcover_colors,
    output_file=f"{site_name}_{month}_{day}_{year}_interactive_map.html",
    csv_file=f"{site_name}_{month}_{day}_{year}_extent_location.csv",
    kml_file=f"{site_name}_{month}_{day}_{year}_extent_location.kml"
)


### Interactive Scatter Plot of Photon Rates with Segment Labels

In [ ]:
import plotly.express as px

def plot_photon_rates_interactive(atl03_data, month, site_name, year, selected_extent=None):
    """
    Create an interactive scatter plot of Terrain Photon Rate vs. Canopy Photon Rate for a specific month.
    
    Args:
        atl03_data (DataFrame): The ATL03 data containing terrain and canopy photon rates.
        month (int): The month to filter the data for (1-12).
        selected_extent (int, optional): The extent ID to highlight. Default is None.
        site_name (str): The name of the site for saving the output file.
        year (int): The year for saving the output file.
    """
    # Filter data for the specified month
    month_data = atl03_data[atl03_data['month'] == month]
    
    # Map landcover descriptions and colors
    month_data['landcover_description'] = month_data['landcover_mode'].map(landcover_descriptions).fillna('Unknown')
    month_data['landcover_color'] = month_data['landcover_mode'].map(landcover_colors).fillna('#d3d3d3')  # Default to gray
    
    # Add a column to identify the selected extent
    month_data['highlight'] = month_data['extent_id'].apply(
        lambda x: 'Selected Extent' if x == selected_extent else 'Others'
    )

    # Customizing hover information
    hover_data = {
        'landcover_mode': True,  # Include the mode value
        'landcover_description': True,  # Show landcover description
        'canopy_photon_rate': ':.2f',  # Format canopy photon rate with 2 decimals
        'terrain_photon_rate': ':.2f',  # Format terrain photon rate with 2 decimals
    }

    # Create scatter plot with landcover coloring and hover data
    fig = px.scatter(
        month_data,
        x='terrain_photon_rate',
        y='canopy_photon_rate',
        color='landcover_description',  # Color points by landcover description
        text='segment_id',  # Use segment_id for brevity in labeling
        title=f'Terrain Photon Rate vs. Canopy Photon Rate for Month {month}',
        width=900,
        height=600,
        labels={'landcover_description': 'Landcover'},
        color_discrete_map=landcover_colors,  # Use landcover colors for points
        hover_name='extent_id',  # Set extent_id as the primary tooltip field
        hover_data=hover_data  # Customize additional tooltip fields
    )

    # Highlight the selected extent
    if selected_extent is not None:
        selected_data = month_data[month_data['extent_id'] == selected_extent]
        if not selected_data.empty:
            fig.add_trace(
                px.scatter(
                    selected_data,
                    x='terrain_photon_rate',
                    y='canopy_photon_rate',
                 
                    color_discrete_sequence=['red'],  # Highlight selected extent in red
                ).data[0]
            )
            for _, row in selected_data.iterrows():
                fig.add_annotation(
                    x=row['terrain_photon_rate'],
                    y=row['canopy_photon_rate'],
                    text=f"Extent: {row['extent_id']}",
                    showarrow=True,
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="red",
                    bgcolor="white",
                    font=dict(size=10, color="black")
                )

    # Update layout for improved aesthetics
    fig.update_layout(
        xaxis_title='Terrain Photon Rate',
        yaxis_title='Canopy Photon Rate',
        title_font_size=18,
        title_x=0.5,  # Center the title
        template='plotly_white',  # Use a clean white theme
        font=dict(size=12),
        legend_title='Landcover',
    )
    
    # Show the plot
    fig.show()

    # Save the interactive plot as an HTML file
    output_filename = f"{site_name}_{year}_month_{month}_interactive_plot.html"
    fig.write_html(output_filename)
    print(f"Interactive plot saved as {output_filename}")


plot_photon_rates_interactive(
    atl03_data, 
    month, 
    site_name, 
    year, 
    selected_extent=2125700283618033664
)


# ICESat-2 Photon Analysis and Visualization

compute and visualize the distribution of relative heights for ground and canopy returns. This section performs several steps, including data filtering, grouping by distance, and calculating moving averages. It also generates plots for both the pseudo wave form, ground and canopy photon classifications,and Terrain Photon Rate vs. Canopy Photon Rate for segment saving them into a single PDF file.

In [ ]:
# Define colors and class labels for each ATL08 class
colors = {
    2: ['green', 'canopy'], 
    3: ['lime', 'canopy_top'], 
    1: ['brown', 'ground']
}

## Data Preparation Functions

Functions that will be used to prepare the data for visualization. These functions will add date-related columns, group data by distance, apply a rolling window, and calculate relative height.


In [ ]:

# Group by 'extent_id' and 'x_atc' to define each photon group
def get_photons_per_set(df, distance_window=5):
    """
    Group data by 'x_atc' distance window, creating sets based on distance (e.g., 5 meters per window) 
    and considering unique 'extent_id'.
    """
    df = df.sort_values(by=['extent_id', 'x_atc'])  # Ensure sorted by extent and along-track distance
    df['distance_group'] = df.groupby('extent_id')['x_atc'].transform(lambda x: (x // distance_window).astype(int))
    return df

# Apply rolling window (moving average) for ground height data
def moving_window_distance(df, window_size=3, min_photons=2):
    """
    Calculate average ground height using a moving window on 'distance_group', ensuring enough ground photons.
    """
    # Filter for ground photons
    ground_df = df[df['atl08_class'] == 1].copy()  # Ground photons only

    # Apply rolling average calculation for ground heights
    ground_df['avg_ground_height'] = ground_df.groupby(['extent_id', 'distance_group'])['height'].transform(
        lambda x: x.rolling(window=window_size, min_periods=min_photons).mean()
    )

    # Handle groups with insufficient photons
    ground_df['avg_ground_height'] = ground_df.groupby(['extent_id', 'distance_group'])['height'].transform(
        lambda x: np.nan if len(x) < min_photons else x.mean()
    )

    # Fill NaN values using forward and backward fills
    ground_df['avg_ground_height'] = ground_df['avg_ground_height'].fillna(method='ffill').fillna(method='bfill')

    # Merge back to full DataFrame
    df = pd.merge(df, ground_df[['extent_id', 'distance_group', 'avg_ground_height']], 
                  on=['extent_id', 'distance_group'], how='left')
    return df

# Compute relative height by subtracting average ground height
def calculate_relative_height(df):
    """
    Subtract average ground height from each photon to compute relative height.
    """
    df['relative_height'] = df['height'] - df['avg_ground_height']
    df['relative_height'] = df['relative_height'].fillna(0)  # Replace NaNs with 0 for cleaner output
    return df
# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# def generate_foliage_profiles_pdf(atl03_data, site_name, year, month, colors):
#     """
#     Generate foliage profile plots for each segment and save them into a PDF.

#     Args:
#         atl03_data (DataFrame): The ATL03 photon data.
#         site_name (str): Name of the study site (used in filenames).
#         year (int): Year of the data.
#         month (int): Month of the data.
#         colors (dict): Dictionary mapping photon classes to colors for classification plots.
#     """
#     # Filter data for the specified month
#     monthly_data = atl03_data[atl03_data['month'] == month]

#     # Define the output PDF file name
#     pdf_filename = f"{site_name}_{year}_{month}_foliage_profiles.pdf"

#     # Create a PdfPages object to save the plots into a single PDF
#     with PdfPages(pdf_filename) as pdf:
#         # Use tqdm progress bar for extents
#         for extent_id in tqdm(monthly_data['extent_id'].unique(), desc="Processing Extents"):
#             extent_data = monthly_data[monthly_data['extent_id'] == extent_id]
            
#             if extent_data.empty:
#                 continue  # Skip if no data is available for this extent
            
#             # Loop through each segment within the extent
#             for segment_id in extent_data['segment_id'].unique():
#                 segment_data = extent_data[extent_data['segment_id'] == segment_id]
                
#                 if segment_data.empty:
#                     continue  # Skip if no data is available for this segment
                
#                 # Process segment data
#                 segment_data = get_photons_per_set(segment_data, distance_window=5)  # 5-meter window
#                 segment_data = moving_window_distance(segment_data, window_size=3, min_photons=2)
#                 segment_data = calculate_relative_height(segment_data)

#                 # Calculate landcover for the segment
#                 canopy_rate = segment_data['canopy_photon_rate'].mean()
#                 terrain_rate = segment_data['terrain_photon_rate'].mean()
#                 landcover = canopy_rate / (terrain_rate + canopy_rate) if (terrain_rate + canopy_rate) > 0 else 0

#                 # Get date-related information for this segment
#                 year = segment_data['year'].iloc[0]
#                 month = segment_data['month'].iloc[0]
#                 day = segment_data['day'].iloc[0]

#                 # Total photon counts
#                 total_photons = len(segment_data)
#                 total_ground_photons = len(segment_data[segment_data['atl08_class'] == 1])
#                 total_canopy_photons = len(segment_data[segment_data['atl08_class'].isin([2, 3])])

#                 # Create a figure with three subplots (1 row, 3 columns)
#                 fig, axes = plt.subplots(1, 3, figsize=(24, 6))

#                 # Subplot 1: Foliage Profile
#                 bin_size = 0.5  # Bin size in meters
#                 bins = np.arange(segment_data['relative_height'].min(), 
#                                  segment_data['relative_height'].max() + bin_size, bin_size)

#                 # Plot Ground Returns
#                 ground_data = segment_data[segment_data['atl08_class'] == 1]
#                 if not ground_data.empty:
#                     counts, edges = np.histogram(ground_data['relative_height'], bins=bins)
#                     normalized_counts = counts / len(segment_data)
#                     mid_points = (edges[:-1] + edges[1:]) / 2
#                     axes[0].plot(normalized_counts, mid_points, label='Ground', color='blue', marker='o')

#                 # Plot Canopy Returns
#                 canopy_data = segment_data[segment_data['atl08_class'].isin([2, 3])]
#                 if not canopy_data.empty:
#                     counts, edges = np.histogram(canopy_data['relative_height'], bins=bins)
#                     normalized_counts = counts / len(segment_data)
#                     mid_points = (edges[:-1] + edges[1:]) / 2
#                     axes[0].plot(normalized_counts, mid_points, label='Canopy', color='green', marker='o')

#                 # Foliage Profile Plot Settings
#                 axes[0].set_xlabel('Normalized Count Density')
#                 axes[0].set_ylabel('Height Relative to Ground (m)')
#                 axes[0].set_title(f'Segment {segment_id} Extent {extent_id} ({month}-{day}-{year})\n'
#                                   f'Total: {total_photons} | Ground: {total_ground_photons} | Canopy: {total_canopy_photons}\n'
#                                   f'Landcover: {landcover:.2f}')
#                 axes[0].legend(title='Class')
#                 axes[0].grid(True)

#                 # Subplot 2: Photon Classification
#                 d0 = np.min(segment_data['segment_dist'])  # Adjust x_atc
#                 for class_val, (color, label) in colors.items():
#                     subset = segment_data[segment_data['atl08_class'] == class_val]
#                     axes[1].plot(subset['segment_dist'] + subset['x_atc'] - d0, 
#                                  subset['height'], 'o', markersize=1, color=color, label=label)

#                 # Classification Plot Settings
#                 axes[1].set_xlabel('$x_{ATC}$ (m)')
#                 axes[1].set_ylabel('Height (m)')
#                 axes[1].set_title(f'Photon Classification for Segment {segment_id} ({month}-{day}-{year})\n'
#                                   f'Total: {total_photons} | Ground: {total_ground_photons} | Canopy: {total_canopy_photons}')
#                 axes[1].legend(title='Class')
#                 axes[1].grid(True)

#                 # Subplot 3: Terrain vs. Canopy Photon Rate
#                 monthly_data['highlight'] = monthly_data['extent_id'].apply(
#                     lambda x: 'red' if x == extent_id else 'blue'
#                 )
#                 axes[2].scatter(
#                     monthly_data['terrain_photon_rate'],
#                     monthly_data['canopy_photon_rate'],
#                     c=monthly_data['highlight'],
#                     alpha=0.6,
#                     edgecolor='w',
#                     s=20
#                 )
#                 axes[2].set_xlabel('Terrain Photon Rate')
#                 axes[2].set_ylabel('Canopy Photon Rate')
#                 axes[2].set_title('Terrain Photon Rate vs. Canopy Photon Rate\n'
#                                   f'Highlighting Extent {extent_id}')
#                 axes[2].grid(True)

#                 # Save the current figure to the PDF
#                 pdf.savefig()  # Save figure into the PDF
#                 plt.close(fig)  # Close the figure to avoid overlap

#     print(f"PDF saved as {pdf_filename}")


In [ ]:
%%time
def generate_foliage_profiles_pdf(atl03_data, site_name, year, month, colors, max_pages_per_pdf=50):
    """
    Generate foliage profile plots for each segment and save them into multiple PDFs with a page limit.

    Args:
        atl03_data (DataFrame): The ATL03 photon data.
        site_name (str): Name of the study site (used in filenames).
        year (int): Year of the data.
        month (int): Month of the data.
        colors (dict): Dictionary mapping photon classes to colors for classification plots.
        max_pages_per_pdf (int): Maximum number of pages per PDF.
    """
    # Filter data for the specified month
    monthly_data = atl03_data[atl03_data['month'] == month]

    # Get unique extent and segment pairs
    extent_segment_pairs = [(extent_id, segment_id)
                            for extent_id in monthly_data['extent_id'].unique()
                            for segment_id in monthly_data[monthly_data['extent_id'] == extent_id]['segment_id'].unique()]

    # Split the extent-segment pairs into chunks of `max_pages_per_pdf`
    chunks = [extent_segment_pairs[i:i + max_pages_per_pdf] for i in range(0, len(extent_segment_pairs), max_pages_per_pdf)]

    # Generate one PDF per chunk
    for i, chunk in enumerate(chunks):
        pdf_filename = f"{site_name}_{year}_{month}_foliage_profiles_part{i + 1}.pdf"

        with PdfPages(pdf_filename) as pdf:
            for extent_id, segment_id in tqdm(chunk, desc=f"Generating {pdf_filename}"):
                segment_data = monthly_data[(monthly_data['extent_id'] == extent_id) &
                                            (monthly_data['segment_id'] == segment_id)]
                
                if segment_data.empty:
                    continue  # Skip if no data is available for this segment
                
                # Process and prepare the segment data
                segment_data = get_photons_per_set(segment_data, distance_window=5)
                segment_data = moving_window_distance(segment_data, window_size=3, min_photons=2)
                segment_data = calculate_relative_height(segment_data)

                # Create a figure with three subplots
                fig, axes = plt.subplots(1, 3, figsize=(24, 6))

                # Subplot 1: Foliage Profile
                bin_size = 0.5
                bins = np.arange(segment_data['relative_height'].min(),
                                 segment_data['relative_height'].max() + bin_size, bin_size)
                
                # Plot ground returns
                ground_data = segment_data[segment_data['atl08_class'] == 1]
                if not ground_data.empty:
                    counts, edges = np.histogram(ground_data['relative_height'], bins=bins)
                    normalized_counts = counts / len(segment_data)
                    mid_points = (edges[:-1] + edges[1:]) / 2
                    axes[0].plot(normalized_counts, mid_points, label='Ground', color='blue', marker='o')

                # Plot canopy returns
                canopy_data = segment_data[segment_data['atl08_class'].isin([2, 3])]
                if not canopy_data.empty:
                    counts, edges = np.histogram(canopy_data['relative_height'], bins=bins)
                    normalized_counts = counts / len(segment_data)
                    mid_points = (edges[:-1] + edges[1:]) / 2
                    axes[0].plot(normalized_counts, mid_points, label='Canopy', color='green', marker='o')

                # Foliage profile settings
                axes[0].set_xlabel('Normalized Count Density')
                axes[0].set_ylabel('Height Relative to Ground (m)')
                axes[0].set_title(f'Segment {segment_id} Extent {extent_id}')
                axes[0].legend(title='Class')
                axes[0].grid(True)

                # Subplot 2: Photon Classification
                d0 = np.min(segment_data['segment_dist'])
                for class_val, (color, label) in colors.items():
                    subset = segment_data[segment_data['atl08_class'] == class_val]
                    axes[1].plot(subset['segment_dist'] + subset['x_atc'] - d0, 
                                 subset['height'], 'o', markersize=1, color=color, label=label)
                
                # Classification settings
                axes[1].set_xlabel('$x_{ATC}$ (m)')
                axes[1].set_ylabel('Height (m)')
                axes[1].set_title(f'Photon Classification')
                axes[1].legend(title='Class')
                axes[1].grid(True)

                # Subplot 3: Terrain vs. Canopy Photon Rate
                monthly_data['highlight'] = monthly_data['extent_id'].apply(
                    lambda x: 'red' if x == extent_id else 'blue'
                )
                axes[2].scatter(
                    monthly_data['terrain_photon_rate'],
                    monthly_data['canopy_photon_rate'],
                    c=monthly_data['highlight'],
                    alpha=0.6,
                    edgecolor='w',
                    s=20
                )
                axes[2].set_xlabel('Terrain Photon Rate')
                axes[2].set_ylabel('Canopy Photon Rate')
                axes[2].set_title('Terrain vs. Canopy Photon Rate')
                axes[2].grid(True)

                # Save the figure into the current PDF
                pdf.savefig()
                plt.close(fig)
        
        print(f"Saved {pdf_filename} with up to {max_pages_per_pdf} pages.")

In [ ]:
generate_foliage_profiles_pdf(atl03_data, site_name, year,month,colors=colors)